# Conversión de modelos de cuadripolos
<img src="./logo_UTN.svg" align="right" width="150" /> 

#### Por Mariano Llamedo Soria

### Resumen 
En este documento se presentan algunas de las capacidades que posee el módulo PyTC2 para operar con cuadripolos. Se muestran ejemplos de cómo definir redes y sus modelos Z, Y y $T_{ABCD}$ asociados, como también algunas redes que implementan dichos modelos.

* Funciones de dibujo de redes: [dibujar_Pi](), [dibujar_Tee]()
* Funciones de conversión y definición de cuadripolos: [Z2Tabcd_s](), [Y2Tabcd_s](), [TabcdY_s](), [TabcdZ_s](), [TabcdLZY_s](), 
* Funciones para presentación de markdown y latex: [print_latex](), [print_subtitle](), [a_equal_b_latex_s]()


### Introducción

A lo largo del curso se presentó una metodología sistemática para arribar a una función transferencia $T(s)$ a partir de restricciones de la función de módulo $\vert T(j\omega) \vert $ o retardo $ \tau(\omega) $. Si bien en primera instancia arribamos a una $T_{LP}(s)$ pasabajos, es posible mediante núcleos de transformación el pasaje a otro tipo de transferencias (pasa-alto, pasabanda, etc).




In [5]:
from pytc2.cuadripolos import *
from pytc2.general import print_latex, print_subtitle, a_equal_b_latex_s
import sympy as sp

y11, y12, y21, y22 = sp.symbols('y11, y12, y21, y22', complex=True)
z11, z12, z21, z22 = sp.symbols('z11, z12, z21, z22', complex=True)
A, B, C, D = sp.symbols('A, B, C, D', complex=True)
Ai, Bi, Ci, Di = sp.symbols('Ai, Bi, Ci, Di', complex=True)
h11, h12, h21, h22 = sp.symbols('h11, h12, h21, h22', complex=True)
g11, g12, g21, g22 = sp.symbols('g11, g12, g21, g22', complex=True)
v1, v2, i1, i2 = sp.symbols('v1, v2, i1, i2', complex=True)

# Parámetros Z (impedancia - circ. abierto)
ZZ = sp.Matrix([[z11, z12], [z21, z22]])
# vars. dependientes
vv = sp.Matrix([[v1], [v2]])
# vars. INdependientes
ii = sp.Matrix([[i1], [i2]])

# Parámetros Y (admitancia - corto circ.)
YY = sp.Matrix([[y11, y12], [y21, y22]])
# vars. dependientes
# ii = sp.Matrix([[i1], [i2]])
# vars. INdependientes
# vv = sp.Matrix([[v1], [v2]])

# Parámetros H (híbridos h)
HH = sp.Matrix([[h11, h12], [h21, h22]])
# vars. dependientes
h_dep = sp.Matrix([[v1], [i2]])
# vars. INdependientes
h_ind = sp.Matrix([[i1], [v2]])

# Parámetros G (híbridos g)
GG = sp.Matrix([[g11, g12], [g21, g22]])
# vars. dependientes
g_dep = sp.Matrix([[i1], [v2]])
# vars. INdependientes
g_ind = sp.Matrix([[v1], [i2]])

# Parámetros Tabcd (Transmisión, ABCD)
TT = sp.Matrix([[A, -B], [C, -D]])
# vars. dependientes
t_dep = sp.Matrix([[v1], [i1]])
# vars. INdependientes.  (Signo negativo de corriente)
t_ind = sp.Matrix([[v2], [i2]])

# Parámetros Tdcba (Transmisión inversos, DCBA)
TTi = sp.Matrix([[Ai, Bi], [-Ci, -Di]])
# vars. dependientes
ti_dep = sp.Matrix([[v2], [i2]])
# vars. INdependientes. (Signo negativo de corriente)
ti_ind = sp.Matrix([[v1], [i1]])

# Diccionario con la definición de cada modelo
model_dct = [ { 'model_name': 'Z', 'matrix': ZZ, 'dep_var': vv, 'indep_var':ii },
              { 'model_name': 'Y', 'matrix': YY, 'dep_var': ii, 'indep_var':vv },
              { 'model_name': 'H', 'matrix': HH, 'dep_var': h_dep, 'indep_var':h_ind },
              { 'model_name': 'G', 'matrix': GG, 'dep_var': g_dep, 'indep_var':g_ind },
              { 'model_name': 'T', 'matrix': TT, 'dep_var': t_dep, 'indep_var':t_ind, 'neg_i2_current': True },
              { 'model_name': 'Ti', 'matrix': TTi, 'dep_var': ti_dep, 'indep_var':ti_ind, 'neg_i2_current': True}
            ]


In [6]:
import os


str_table = '$ \\begin{array}{ l ' + ' c'*len(model_dct) + ' }' + os.linesep 

for src_model in model_dct:
    str_table +=  ' & ' + src_model['model_name']

str_table = str_table + ' \\\\ ' + os.linesep

for dst_model in model_dct:

    str_table +=   dst_model['model_name']   + ' & '
    
    for src_model in model_dct:
        
        HH_z = Model_conversion( src_model, dst_model )

        str_table +=  sp.latex( HH_z['matrix'] )  + ' & '
        
    str_table = str_table[:-2] + ' \\\\ ' + os.linesep
    
    #print_latex( str_table )
    
str_table = str_table[:-4] + os.linesep + '\\end{array} $'

print_latex( str_table )
        

<IPython.core.display.Math object>

In [10]:

# Parámetros Imagen
gamma, z1, z2 = sp.symbols('\gamma, z1, z2', complex=True)

Tim = I2Tabcd_s(gamma, z1, z2)
 #(Signo negativo de corriente)
Tim[:,1] = Tim[:,1] * -1
# vars. dependientes
tim_dep = t_dep
# vars. INdependientes.
tim_ind = t_ind

# Diccionario con la definición de cada modelo
model_dct = [ { 'model_name': 'Im', 'matrix': Tim, 'dep_var': tim_dep, 'indep_var':tim_ind, 'neg_i2_current': True }, # T_ABCD
              { 'model_name': 'Z', 'matrix': ZZ, 'dep_var': vv, 'indep_var':ii },
              { 'model_name': 'Y', 'matrix': YY, 'dep_var': ii, 'indep_var':vv }
            ]


str_table = '$ \\begin{array}{ l ' + ' c'*len(model_dct) + ' }' + os.linesep 

for src_model in model_dct:
    str_table +=  ' & ' + src_model['model_name']

str_table = str_table + ' \\\\ ' + os.linesep

for dst_model in model_dct:

    str_table +=   dst_model['model_name']   + ' & '
    
    for src_model in model_dct:
        
        HH_z = Model_conversion( src_model, dst_model )

        str_table +=  sp.latex( HH_z['matrix'] )  + ' & '
        
    str_table = str_table[:-2] + ' \\\\ ' + os.linesep
    
    #print_latex( str_table )
    
str_table = str_table[:-4] + os.linesep + '\\end{array} $'

print_latex( str_table )
        

<IPython.core.display.Math object>

In [13]:


print(  sp.simplify(sp.det(Tim)) )


-sqrt(z2/z1)*sqrt(z1/z2)*cosh(\gamma)**2 + cosh(\gamma)**2 - 1
